Задание 1.1

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной KEYWORDS. Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>.

Задание 2.1

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

KEYWORDS = ['разработчик']

titles_with_keywords = pd.DataFrame()

# обращаемся к странице
req = requests.get('https://habr.com/ru/all/')

# получаем html страницы
soup = BeautifulSoup(req.text)

# ищем посты
posts = soup.find_all('article', class_='tm-articles-list__item')

# ищем ключевые слова в постах
for post in posts: 
    title = post.find_all('a', class_='tm-article-snippet__title-link')[0].span.text
    
#     проверяем, есть ли ключевое слово в заголовке
    if not any([(keyword in title) for keyword in KEYWORDS]):
        continue
    
    date = pd.to_datetime(post.find_all('span', class_= 'tm-article-snippet__datetime-published')[0].time.get('title'))
    link = 'https://habr.com' + post.find_all('a', class_='tm-article-snippet__title-link')[0].get('href') 
    
#     получаем html статей
    soup_with_links = BeautifulSoup(requests.get(link).text)
    
    text = soup_with_links.find_all('div', class_='article-formatted-body article-formatted-body_version-1')
    
    row = {'дата': date, 'заголовок': title, 'ссылка': link, 'текст_статьи': text}
    
# пополняем датафрем
    titles_with_keywords = pd.concat([titles_with_keywords, pd.DataFrame([row])])
    
titles_with_keywords

,дата,заголовок,ссылка,текст_статьи
0,2021-10-07 19:14:00,Каждому по проекту: как разработчику начать ра...,https://habr.com/ru/post/582254/,"[[[<img alt=""image"" data-blurred=""true"" data-s..."


Задание 1.2

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком. Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>.

In [10]:
EMAIL = ["achris3007@gmail.com", "a_chris@mail.ru"]

URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

headers = {
    'Accept': 'application/json, text/plain, */',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Content-Length': '43',
    'Content-Type': 'application/json;charset=UTF-8',
    'Host': 'identityprotection.avast.com',
    'Origin': 'https://www.avast.com',
    'Referer': 'https://www.avast.com/',
    'sec-ch-ua': '"Chromium";v="94", "Google Chrome";v="94", ";Not A Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36',
    'Vaar-Header-App-Build-Version': '1.0.0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Vaar-Version': '0'
}

params = {
    'emailAddresses': EMAIL
}

res = requests.post(URL, json=params, headers=headers)
email_leaks = res.json()
email_leaks

{'breaches': {'5': {'breachId': 5,
   'site': 'twitter.com',
   'recordsCount': 27329821,
   'description': "Login credentials for over 32 Million Twitter users were discovered for sale on a dark web marketplace for a price of 10 bitcoin each. It's widely suspected that these credentials weren't the result of a breach of Twitter itself, but rather of information gleaned directly from botnet infected hosts.",
   'publishDate': '2016-10-26T00:00:00Z',
   'statistics': {'usernames': 0, 'passwords': 26847787, 'emails': 27329821}},
  '16659': {'breachId': 16659,
   'site': 'livejournal.com',
   'recordsCount': 30485836,
   'description': "In 2017, social network LiveJournal's database was allegedly breached. The stolen data contains passwords, email addresses and additional personal information. This breach is being privately shared in the dark web.",
   'publishDate': '2019-05-23T00:00:00Z',
   'statistics': {'usernames': 30485732,
    'passwords': 29634403,
    'emails': 30482018}}},
 'da

In [11]:
df = pd.DataFrame()

for key in email_leaks['breaches']:
    site = email_leaks['breaches'][key]['site']
    date = email_leaks['breaches'][key]['publishDate']
    description = email_leaks['breaches'][key]['description']
    
    df = pd.concat([df, pd.DataFrame({'дата утечки': [date], 'источник утечки': [site], 'описание утечки': [description]})])
    
df

,дата утечки,источник утечки,описание утечки
0,2016-10-26T00:00:00Z,twitter.com,Login credentials for over 32 Million Twitter ...
0,2019-05-23T00:00:00Z,livejournal.com,"In 2017, social network LiveJournal's database..."


Задание 2.2

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте. В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.

In [7]:
WALL_GET = 'https://api.vk.com/method/wall.get'
VERSION = '5.84'
TOKEN = '123'
GROUP = 'netology'

params = {
    'access_token': TOKEN,
    'v': VERSION,
    'domain': GROUP,
    'count': 50
}

res = requests.get(WALL_GET, params)
res.json()

df = pd.DataFrame(res.json()['response']['items'])

df_new = pd.DataFrame({'дата поста': df['date'], 'текст поста': df['text']})
df_new

,дата поста,текст поста
0,1633532400,"Считать налоги — важный навык для всех, кто тр..."
1,1633622760,"Многим хоть раз казалось, что их успех не засл..."
2,1633591260,И снова видео прямо здесь во ВКонтакте 🙃\n\nРа...
3,1633508640,15 октября стартует бесплатный курс-симулятор ...
4,1633446660,Частые спутники современного образа жизни — тр...
5,1633420800,Открыли новое направление Lifestyle & Hobby 🧘‍...
6,1633360920,"Проверка юзабилити, корпоративный блог, общени..."
7,1633332840,👨‍💻 26 октября стартует бесплатный курс «Первы...
8,1633248120,"Задумывались когда-нибудь о том, какие организ..."
9,1633186801,"Делимся подборкой событий, в которых участвуем..."
